# Implementing Custom Tests

## Implementing custom metrics and threshold tests

Custom metrics allow you to extend the default set of metrics provided by ValidMind and provide full flexibility for documenting any type of model or use case. Metrics and threshold tests are similar in that they both provide a way to evaluate a model. The difference is that metrics capture any arbitrary set of values that measure a behavior in a dataset(s) or model(s), while threshold tests are Boolean tests that evaluate whether a behavior passes or fails a set of criteria.

### Documentation components of a metric and threshold test

A **metric** is composed of the following documentation elements:

- Title
- Description
- Results Table(s)
- Plot(s)

A **threshold test** is composed of the following documentation elements:

- Title
- Description
- Test Parameters
- Results Table(s)
- Plot(s)

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install --upgrade validmind

Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [2]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    api_secret="API_SECRET",
    project="xxxxxxxxxxxxxxxxxxxxxxxxx"
)

2023-07-11 09:06:23,379 - INFO - api_client - Connected to ValidMind. Project: Customer Churn Model - Test for Notebooks, Sydney (cljxccag7000016mkriy2l77l)


### Metric class signature

In order to implement a custom metric or threshold test, you must create a class that inherits from the `Metric` or `ThresholdTest` class. The class signatures below show the different methods that need to be implemented in order to provide the required documentation elements:

```python
@dataclass
class ExampleMetric(Metric):
    name = "mean_of_values"

    # Markdown compatible description of the metric
    def description(self):

    # Code to compute the metric and cache its results and Figures
    def run(self):

    # Code to build a list of ResultSummaries that form the results tables
    def summary(self, metric_values):
```

We'll now implement a sample metric to illustrate their different documentation components.

### Implementing a custom metric

The following example shows how to implement a custom metric that calculates the mean of a list of numbers.

#### Basic metric implementation

At its most basic, a metric implementation requires a `run()` method that computes the metric and caches its results and Figures. The run() method is called by the ValidMind client when the metric is executed. The `run()` should return any value that can be serialized to JSON.

In the example below we also provide a simple description for the metric:

In [3]:
from dataclasses import dataclass
from validmind.vm_models import Metric

@dataclass
class MeanMetric(Metric):
    name = "mean_of_values"

    def description(self):
        return "Calculates the mean of the provided values"

    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        
        values = self.params["values"]
        mean = sum(values) / len(values)
        
        return self.cache_results(mean)

#### Testing the custom metric

We should run a metric first without running an entire test plan and test its behavior.

The only requirement to run a metric is build a `TestContext` object and pass it to the metric initializer. Test context objects allow metrics and tests to access data inside their class methods in a predictable way. By default, ValidMind provides support for the following special keys in a test context objects:

- `dataset`
- `model`
- `models`

When a test context object is build with one of these keys, the corresponding value is automatically added to the object as an attribute. For example, if you build a test context object with the `dataset` key, you can access the dataset inside the metric's `run()` method as `self.dataset`. We'll illustrate this in detail in the next section.

In our simple example, we don't need to pass any arguments to the `TestContext` initializer.

In [4]:
from validmind.vm_models.test_context import TestContext

test_context = TestContext()
mean_metric = MeanMetric(test_context=test_context, params={
    "values": [1, 2, 3, 4, 5]
})
mean_metric.run()

TestPlanMetricResult(result_id="mean_of_values", metric, figures)

You can also inspect the results of the metric by accessing the `result` variable:

In [5]:
mean_metric.result.show()

### Add a `summary()` method to the custom metric

The `summary()` method is used to build a `ResultSummary` object that can display the results of our test as a list of one or more summray tables. The `ResultSummary` class takes a `results` argument that is a list of `ResultTable` objects.

Each `ResultTable` object is composed of a `data` and `metadata` attribute. The `data` attribute is any valid Pandas tabular DataFrame and `metadata` is a `ResultTableMetadata` instance that takes `title` as the table description.

In [6]:
from dataclasses import dataclass

import pandas as pd
from validmind.vm_models import Metric, ResultSummary, ResultTable, ResultTableMetadata

@dataclass
class MeanMetric(Metric):
    name = "mean_of_values"

    def description(self):
        return "Calculates the mean of the provided values"

    def summary(self, metric_value):
        # Create a dataframe structure that can be rendered as a table
        simple_df = pd.DataFrame({"Mean of Values": [metric_value]})

        return ResultSummary(
            results=[
                ResultTable(
                    data=simple_df,
                    metadata=ResultTableMetadata(title="Example Table"),
                ),                
            ]
        )        
        
    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        
        values = self.params["values"]
        mean = sum(values) / len(values)
        return self.cache_results(mean)

In [7]:
from validmind.vm_models.test_context import TestContext

test_context = TestContext()
mean_metric = MeanMetric(test_context=test_context, params={
    "values": [1, 2, 3, 4, 5]
})
mean_metric.run()

TestPlanMetricResult(result_id="mean_of_values", metric, figures)

In [8]:
mean_metric.result.show()

### Add figures to a metric

You can also add figures to a metric by passing a `figures` list to `cache_results()`. Each figure is a `Figure` object that takes the following arguments:

- `for_object`: The name of the object that the figure is for. Usually defaults to `self`
- `figure`: A Matplotlib or Plotly figure object
- `key`: A unique key for the figure

The developer framework uses `for_object` and `key` to link figures to the corresponding metric or test.

In [9]:
from dataclasses import dataclass

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from validmind.vm_models import Figure, Metric, ResultSummary, ResultTable, ResultTableMetadata

@dataclass
class MeanMetric(Metric):
    name = "mean_of_values"

    def description(self):
        return "Calculates the mean of the provided values"

    def summary(self, metric_value):
        # Create a dataframe structure that can be rendered as a table
        simple_df = pd.DataFrame({"Mean of Values": [metric_value]})

        return ResultSummary(
            results=[
                ResultTable(
                    data=simple_df,
                    metadata=ResultTableMetadata(title="Example Table"),
                ),
            ]
        )        

        
    def run(self):
        if "values" not in self.params:
            raise ValueError("values must be provided in params")

        if not isinstance(self.params["values"], list):
            raise ValueError("values must be a list")
        
        values = self.params["values"]
        mean = sum(values) / len(values)

        # Create a random histogram with matplotlib
        fig, ax = plt.subplots()
        ax.hist(np.random.randn(1000), bins=20, color="blue")
        ax.set_title("Histogram of random numbers")
        ax.set_xlabel("Value")
        ax.set_ylabel("Frequency")

        # Do this if you want to prevent the figure from being displayed
        plt.close("all")
        
        figure = Figure(
            for_object=self,
            key=self.key,
            figure=fig
        )

        return self.cache_results(mean, figures=[figure])

In [10]:
from validmind.vm_models.test_context import TestContext

test_context = TestContext()
mean_metric = MeanMetric(test_context=test_context, params={
    "values": [1, 2, 3, 4, 5]
})
mean_metric.run()

TestPlanMetricResult(result_id="mean_of_values", metric, figures)

In [11]:
mean_metric.result.show()

In [12]:
from validmind.vm_models import TestPlan

class MyCustomTestPlan(TestPlan):
    """
    Custom test plan
    """

    name = "my_custom_test_plan"
    required_context = []
    tests = [MeanMetric]

my_custom_test_plan = MyCustomTestPlan(config={
    "mean_of_values": {
        "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    },
})
results = my_custom_test_plan.run()

AttributeError: type object 'MeanMetric' has no attribute 'split'